In [2]:
import os
import sys
sys.path.append('../')

import numpy as np
import matplotlib
from matplotlib.dates import DateFormatter
import matplotlib.pyplot as plt
import datetime
import pandas as pd
from pprint import pprint
from scipy.optimize import curve_fit

import src.io as sio
import src.preprocessing as spp
import src.fitting as sft
import ipympl
import pytz

In [3]:
AFM_FOLDER = sio.get_folderpath("20201030_Bakeout")

In [6]:
df = sio.read_tm224_data("temperature-monitoring04.xls", AFM_FOLDER)

fig, ax = plt.subplots()

myFmt = DateFormatter("%H:%M\n%a")
ax.xaxis.set_major_formatter(myFmt)

ax.plot(df["MPL_datetimes"], df["Baseplate"])

x, y = sft.time_extrapolation(df, end_date="11-Nov-20 10:00", start_index=2500)
ax.plot(x, y)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …